In [1]:
import random
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

/home/kiwoongyoon/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [2]:

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../DATA/train.csv'

In [5]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [10]:
price_data

,corp,loca,product,product_loca,item,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,...,2023-02-22,2023-02-23,2023-02-24,2023-02-25,2023-02-26,2023-02-27,2023-02-28,2023-03-01,2023-03-02,2023-03-03
0,A,J,TG,8,TG A J,0.0,0.0,1728.0,1408.0,1250.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,A,S,TG,9,TG A S,0.0,0.0,2526.0,2134.0,2075.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,B,J,TG,8,TG B J,0.0,0.0,1692.0,1516.0,1471.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0
3,B,S,TG,9,TG B S,0.0,0.0,1944.0,1815.0,1717.0,...,3202.0,3478.0,3939.0,3677.0,0.0,4057.0,3821.0,4037.0,4004.0,4241.0
4,C,J,TG,8,TG C J,0.0,0.0,1965.0,1794.0,1773.0,...,4017.0,4585.0,4835.0,5550.0,0.0,5037.0,2643.0,3742.0,3983.0,5175.0
5,C,S,TG,9,TG C S,0.0,0.0,2078.0,2002.0,1815.0,...,3228.0,3484.0,3773.0,4298.0,0.0,4180.0,4234.0,4357.0,4466.0,4748.0
6,D,J,TG,8,TG D J,0.0,0.0,1616.0,1337.0,1234.0,...,0.0,0.0,1838.0,1829.0,0.0,0.0,0.0,0.0,0.0,0.0
7,D,S,TG,9,TG D S,0.0,0.0,2048.0,1757.0,1719.0,...,2539.0,2955.0,3323.0,3321.0,0.0,3409.0,3236.0,4631.0,4114.0,4146.0
8,E,J,TG,8,TG E J,0.0,0.0,1692.0,1475.0,1392.0,...,3157.0,4793.0,2403.0,1489.0,0.0,2620.0,1673.0,0.0,1904.0,1622.0
9,E,S,TG,9,TG E S,0.0,0.0,2151.0,1829.0,2002.0,...,3400.0,3349.0,3026.0,2946.0,0.0,3418.0,3141.0,4235.0,3960.0,3791.0


In [7]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [9]:


le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])
price_df = price_data.copy()

In [12]:
from RevIN import RevIN

ModuleNotFoundError: No module named 'RevIN'

In [14]:
# Class Definitions


class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()

        # 설정값에서 필요한 파라미터를 가져옵니다.
        self.seq_len = configs.seq_len  # 입력 시퀀스 길이
        self.pred_len = configs.pred_len  # 예측할 시퀀스 길이
        self.enc_in = configs.enc_in  # 입력 데이터의 차원 수(채널 수)
        self.d_model = configs.d_model  # 모델의 차원 수
        self.dropout = configs.dropout  # 드롭아웃 비율

        self.rnn_type = configs.rnn_type  # RNN의 유형 (RNN, GRU, LSTM 중 선택)
        self.dec_way = configs.dec_way  # 예측 방식 (rmf: Recurrent Multi-step Forecasting, pmf: Parallel Multi-step Forecasting 중 선택)
        self.seg_len = configs.seg_len  # 세그먼트의 길이
        self.channel_id = configs.channel_id  # 채널 ID 사용 여부
        self.revin = configs.revin  # RevIN 레이어 사용 여부

        assert self.rnn_type in ['rnn', 'gru', 'lstm']
        assert self.dec_way in ['rmf', 'pmf']

        #입력 시퀀스를 세그먼트로 나누어 세그먼트의 개수를 계산합니다.
        self.seg_num_x = self.seq_len//self.seg_len

        # valueEmbedding 레이어: 세그먼트를 d_model 차원의 벡터로 임베딩합니다.
        self.valueEmbedding = nn.Sequential(
            nn.Linear(self.seg_len, self.d_model),
            nn.ReLU()
        )

        if self.rnn_type == "rnn":
            self.rnn = nn.RNN(input_size=self.d_model, hidden_size=self.d_model, num_layers=1, bias=True,
                              batch_first=True, bidirectional=False)
        elif self.rnn_type == "gru":
            self.rnn = nn.GRU(input_size=self.d_model, hidden_size=self.d_model, num_layers=1, bias=True,
                              batch_first=True, bidirectional=False)
        elif self.rnn_type == "lstm":
            self.rnn = nn.LSTM(input_size=self.d_model, hidden_size=self.d_model, num_layers=1, bias=True,
                              batch_first=True, bidirectional=False)

        if self.dec_way == "rmf":
            self.seg_num_y = self.pred_len // self.seg_len
            self.predict = nn.Sequential(
                nn.Dropout(self.dropout),
                nn.Linear(self.d_model, self.seg_len)
            )
        elif self.dec_way == "pmf":
            self.seg_num_y = self.pred_len // self.seg_len

            if self.channel_id:
                # seg_num_y 가 28//2 인 14이고 모델 차원이 32이면 14x16의 파라미터 행렬 생성된다. 
                self.pos_emb = nn.Parameter(torch.randn(self.seg_num_y, self.d_model // 2))
                # 채널의개수 x 모델차원//2 의 파라미터 행령 생성된다. 
                self.channel_emb = nn.Parameter(torch.randn(self.enc_in, self.d_model // 2))
            else:
                self.pos_emb = nn.Parameter(torch.randn(self.seg_num_y, self.d_model))

            self.predict = nn.Sequential(
                # 순서대로 처리하게 해주는 역할 
                nn.Dropout(self.dropout),
                nn.Linear(self.d_model, self.seg_len)
            )
        if self.revin:
            self.revinLayer = RevIN(self.enc_in, affine=False, subtract_last=False)


    def forward(self, x):

        # b:batch_size c:channel_size s:seq_len s:seq_len
        # d:d_model w:seg_len n:seg_num_x m:seg_num_y
        batch_size = x.size(0)
        # print(f"batch size : {batch_size}, {x.shape}")
        # print(f"origin input: {x}")
        # normalization and permute     b,s,c -> b,c,s
        if self.revin:
            x = self.revinLayer(x, 'norm').permute(0, 2, 1)
        else:
            seq_last = x[:, -1:, :].detach()
            x = (x - seq_last).permute(0, 2, 1) # b,c,s

        # segment and embedding    b,c,s -> bc,n,w -> bc,n,d
        newx = x.reshape(-1, self.seg_num_x, self.seg_len)
        # print(f"newx size : {newx.shape}")
        x = self.valueEmbedding(newx)
        # print(f"after embedding : {x.shape}")
        # print(f"after embedding input: {x}")
        
        # encoding
        if self.rnn_type == "lstm":_, (hn, cn) = self.rnn(x)
        #hidden layer = (1, batchsize*channel num , d_model(hiddensize))
        else: _, hn = self.rnn(x) # bc,n,d  1,bc,d 
        

        # decoding
        if self.dec_way == "rmf":
            y = []
            for i in range(self.seg_num_y):
                yy = self.predict(hn)    # 1,bc,l
                yy = yy.permute(1,0,2)   # bc,1,l
                y.append(yy)
                yy = self.valueEmbedding(yy)
                if self.rnn_type == "lstm":
                    _, (hn, cn) = self.rnn(yy, (hn, cn))
                else:
                    _, hn = self.rnn(yy, hn)
            y = torch.stack(y, dim=1).squeeze(2).reshape(-1, self.enc_in, self.pred_len) # b,c,s
        
        elif self.dec_way == "pmf":
            if self.channel_id:
                # m,d//2 -> 1,m,d//2 -> c,m,d//2
                # c,d//2 -> c,1,d//2 -> c,m,d//2
                # c,m,d -> cm,1,d -> bcm, 1, d
                # print(f"before pos emb : {self.pos_emb.unsqueeze(0).repeat(self.enc_in, 1, 1).shape}")#(1,14,16)
                # print(f"before channel emb : {self.channel_emb.unsqueeze(1).repeat(1, self.seg_num_y, 1).shape}")#(1,14,16)
                pos_emb = torch.cat([
                    #채널의 개수 enc_in만큼 pos_emb를 쌓아올린다
                    self.pos_emb.unsqueeze(0).repeat(self.enc_in, 1, 1),
                    # (enc_in)x(d_model/2) -> (enc_in,1,d_model/2)->  (enc_in,seg_num_y, d_model/2)
                    self.channel_emb.unsqueeze(1).repeat(1, self.seg_num_y, 1)
                ], dim=-1).view(-1, 1, self.d_model).repeat(batch_size,1,1)
                # dim = -1에 대해 cat하니까 (enc_in,seg_num_y,d_model) 로 형태 변경 
                # (enc_in,seg_num_y,d_model)-> (enc_in*seg_num_y,1,d_model)--repeat-->(batch*enc_in*seg_num_y,1,d_model)
                # print(f"pos emb : {pos_emb.shape}")
            else:
                # m,d -> bcm,d -> bcm, 1, d
                pos_emb = self.pos_emb.repeat(batch_size * self.enc_in, 1).unsqueeze(1)

            # pos_emb: m,d -> bcm,d ->  bcm,1,d
            # hn, cn: 1,bc,d -> 1,bc,md -> 1,bcm,d
            if self.rnn_type == "lstm":
                _, (hy, cy) = self.rnn(pos_emb,
                                       (hn.repeat(1, 1, self.seg_num_y).view(1, -1, self.d_model),
                                        cn.repeat(1, 1, self.seg_num_y).view(1, -1, self.d_model)))
            else:
                #hn은 아까 gru or rnn의 hiddenlayer로 전체 입력에 대한 요약된 정보이다. (1, 1024,32)-> (1,1024, 32*14)
                # print(f"hiddenlayer : {hn.shape}")
                # print(f"hiddenlayer stac : {hn.repeat(1, 1, self.seg_num_y).shape}")
                # print(f"hiddenlayer stacked: {hn.repeat(1, 1, self.seg_num_y).view(1, -1, self.d_model).shape}")
                
                # 초기의 hiddenstate를 hn.repeat(1, 1, self.seg_num_y).view(1, -1, self.d_model)으로 정의하여 rnn에 전달하기 
                _, hy = self.rnn(pos_emb, hn.repeat(1, 1, self.seg_num_y).view(1, -1, self.d_model))
            # 1,bcm,d -> 1,bcm,w -> b,c,s
            y = self.predict(hy).view(-1, self.enc_in, self.pred_len)

        # permute and denorm
        if self.revin:
            y = self.revinLayer(y.permute(0, 2, 1), 'denorm')
        else:
            y = y.permute(0, 2, 1) + seq_last

        return y
    
    
class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [15]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[5:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data (39개의 특산물x 1523일간의 기록)
time_series_data = reshape_data(price_df)

In [19]:
time_series_data

array([[   0.,    0., 1728., ..., 3397., 3195., 3640.],
       [   0.,    0., 2526., ..., 4173., 4219., 4089.],
       [   0.,    0., 1692., ..., 6382., 3558., 3470.],
       ...,
       [   0.,    0.,    0., ..., 3816., 3321., 2939.],
       [   0.,    0.,  478., ...,  621.,  653.,  643.],
       [   0.,    0.,  395., ...,  574.,  523.,  529.]])

In [53]:
# Define the window size, forecast size, and batch size
window_size = 72   
forecast_size = 28
batch_size = 1024
epoch_count = 100
lr = 0.001
min_delta = 0.001
patience = 10

In [12]:
class Configs:
    def __init__(self):
        self.seq_len = 72
        self.pred_len = 28
        self.enc_in = 1
        self.d_model = 32
        self.dropout = 0.2
        self.rnn_type = 'gru'
        self.dec_way = 'pmf'
        self.seg_len = 2
        self.channel_id = True
        self.revin = False

configs = Configs()


# Model Train 

In [54]:

future_predictions_by_id = {}
loss_history = {}
# Iterate through the data by ID
for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
    # Standardizing the data
    mean_ = np.mean(data)
    std_ = np.std(data)
    
    # Check if std_ is zero and handle standardization accordingly
    if std_ == 0:     standardized_data = data
    else:standardized_data = (data - mean_) / std_
    individual_loss_history = []
    
    # Create DataLoader
    train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
    best_loss = float('inf')
    no_improvement_count = 0

    # Training the model
    seg_rnn = Model(configs)
    seg_rnn.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(seg_rnn.parameters(), lr=lr)
    
    print(idx)
    for epoch in range(1, epoch_count + 1):
        loss_list = []
        seg_rnn.train()
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
            optimizer.zero_grad()
            output = seg_rnn(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        
        print(avg_loss)
        if((epoch % 10) == 0):
            avg_loss = np.mean(loss_list)
            individual_loss_history.append(avg_loss)
            if avg_loss + min_delta < best_loss:
                best_loss = avg_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    break

    loss_history[id_val] = individual_loss_history

    # Predicting the future days using the last window of data
    last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
    future_prediction = seg_rnn(last_window_data)

    # Converting the prediction back to the original scale
    if std_ == 0:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy()
    else:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

    # Store the prediction
    future_predictions_by_id[id_val] = future_prediction
# future_predictions_by_id

  0%|          | 0/39 [00:00<?, ?it/s]

0
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
0.6941241025924683
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
1.1240636110305786
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8738240599632263
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8605774939060211
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8304519057273865
0.8129258751869202
0.8129258751869202
0.81292587

In [56]:
future_predictions_by_id

{'TG A J': array([3818.6895, 3781.889 , 3169.1804, 3384.4336, 3304.458 , 3502.6292,
        4157.496 , 3752.1213, 3352.793 , 3094.4128, 3347.24  , 3446.7712,
        3746.0566, 3646.3115, 3548.9097, 3704.2573, 3266.348 , 3170.4282,
        3457.0056, 3629.4233, 4237.0615, 3871.292 , 3454.9   , 3591.0454,
        3661.132 , 3587.6958, 3588.2607, 3452.6538], dtype=float32),
 'TG A S': array([2429.9707, 1809.7773, 2898.5615, 4615.7563, 4532.9795, 5416.8887,
        3730.5918, 3051.336 , 3775.5295, 3356.1985, 5001.4297, 5082.5557,
        5096.1064, 4555.8755, 3865.9822, 3528.9714, 3126.9922, 5072.4663,
        4401.457 , 4702.466 , 3996.7422, 2890.166 , 2257.8823, 3077.1072,
        4114.2363, 5218.9805, 4782.1196, 4322.6367], dtype=float32),
 'TG B J': array([3691.8484, 4048.4248, 4201.9834, 4165.796 , 4014.2085, 4207.504 ,
        3657.7507, 3499.6973, 3866.654 , 3686.2336, 4453.591 , 4592.419 ,
        4655.335 , 4595.4126, 4091.22  , 4427.9565, 3953.3342, 3839.0388,
        4205.484 ,

In [19]:
submit = pd.read_csv('./../../DATA/sample_submission.csv')


In [20]:
findata = []
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    findata.extend(rounded_predictions)

submit['answer'] = findata
submit.loc[submit['answer'] <= 0, 'answer'] = 0
submit.loc[submit.index % 7 == 1, 'answer'] = 0

In [21]:
submit.tail(33)

,ID,answer
1059,CB_F_J_20230327,613.0
1060,CB_F_J_20230328,642.0
1061,CB_F_J_20230329,592.0
1062,CB_F_J_20230330,559.0
1063,CB_F_J_20230331,605.0
1064,RD_F_J_20230304,525.0
1065,RD_F_J_20230305,0.0
1066,RD_F_J_20230306,469.0
1067,RD_F_J_20230307,461.0
1068,RD_F_J_20230308,489.0


In [22]:
submit.to_csv('segrnn3.csv', index=False)